In [1]:
!pip install boto3
!pip install openai

In [2]:
BUCKET = 'coursera-warehouse'
FULL_COURSES_S3 = 'export/genesis/foundation_data_for_gen_ai/foundation_data_gen_ai_full_courses/part-00000-tid-511212793672894789-f0c46935-46b0-42bd-8342-359a7bcb1500-4-1-c000.snappy.parquet'
READINGS_S3 = 'export/genesis/foundation_data_for_gen_ai/foundation_data_gen_ai_course_readings/part-00000-tid-8510619809744727655-13febe15-3ffe-47d6-9162-9565af1e1267-98-1-c000.snappy.parquet'
SAVE_FILENAME = 'readings_enriched.csv'

## Import Data

In [3]:
import boto3
import pandas as pd
from pyarrow.parquet import ParquetDataset
from io import BytesIO

s3 = boto3.resource('s3')
s3_object = s3.Object(BUCKET, READINGS_S3)
file_data = s3_object.get()["Body"].read()

pf = ParquetDataset(BytesIO(file_data)).read_pandas()

df = pf.to_pandas()
df

,atom_id,atom_version_id,course_id,course_branch_id,course_branch_status,is_course_branch_default,course_module_id,course_module_name,course_lesson_id,course_lesson_name,course_module_desc,course_item_id,course_item_name,course_item_type,course_item_type_id,course_module_order_in_branch,course_lesson_order_in_branch,course_item_order_in_branch,reading_content
0,--5iuKT4EeqA8wrK9wez6Q,1,__2jECXpEeiHbw71qx-dqA,__2jECXpEeiHbw71qx-dqA,Live,True,UU3Yk,Модуль 4.Геологические закономерности размещен...,7hzHN,Урок 6. Месторождения андезитоидных вулканичес...,,Y7aJN,Материалы лекции,reading,103,4,21,94,"<co-content><asset id=""hT1-KSxHEeiTdA5yoE99Fg""..."
1,--PG7d-3EeuvphLW3v4-6w,2,1rWmOd-3Eeu-EQpycSBBow,1rWmOd-3Eeu-EQpycSBBow,Live,True,lJvtj,Project Overview,8dDTs,Your Learning Journey,"In this project-based course, you will learn h...",MhGNK,Project Overview,reading,103,1,1,1,"<co-content><heading level=""1"">Welcome to Your..."
2,--e_h9tbEe628RJzOJU3jw,1,yfMErXgQEe2QHRL3YiV0EQ,branch~8sgWU9tbEe6xnw6R70Kdnw,Live,False,kSa73,Maximizando la Inclusión con Simulaciones PhET,zSLcM,Funciones Inclusivas para Diversas Necesidades...,"En este módulo, nos enfocamos en cultivar un e...",jFLGR,Descripción Interactiva,reading,103,3,9,56,"<co-content><text variant=""body1""><strong>Esta..."
3,--gFYqmsEe6IBQ6UwJe7kw,2,UPMEPeRVEe2nnBJXwaSqgQ,UPMEPeRVEe2nnBJXwaSqgQ,Live,True,BETpD,Course Conclusion,xljR4,"TLD System Lead ""Honors"" Material",,PISSZ,AX for Technical Leads (Powerpoint slides),reading,103,11,13,16,"<co-content><asset id=""Bx_F_zSjTd-AqmnWQi-nwA""..."
4,-0JjQMRGEe2wQAq_QaObVw,4,XYgBjZgWEe2L_wpgKckryw,branch~FfdgPZwAEe2a0hItqNBhjQ,Live,False,5ROis,Module 1: Psychosis Fundamentals,8OmeL,"Module 1, Lesson 1: Introduction to the Course","During the first course module, participants w...",GiUO6,Get to Know Your Instructor!,reading,103,1,1,2,"<co-content><text> </text><img assetId=""roMpFN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54925,zy4Qj5FsEeuEiwo7qexJUw,12,cdQYUI44Eeu3ohKUPTd6hQ,branch~e-1HK5T8EeyHuA5cIqn4NQ,Live,False,KNg2M,Advanced Theories and Techniques Behind Artifi...,tQ85l,Understanding the Symbolic Approach to AI,"In this module, you will learn about what info...",qaXM1,Optional Reading: Artificial Intelligence: A G...,reading,103,3,8,46,<co-content><text>If you want to learn more ab...
54926,zycmQ0ZUEe2WFQ5zp46X_w,2,zmLU1xzCEe2yDw48xzb9Sw,branch~ehvs8y4TEe2qswqioZD8GQ,Live,False,xztfq,Introduction to careers in sustainability,Q5yCf,Welcome!,Welcome to the first step in your journey towa...,ZFjPe,Accessing text-only transcripts,reading,103,1,1,3,<co-content><text>F​or every video in this cou...
54927,zzYRQQOHEe-7ow448vBLiw,1,7qmj5qhIEe2txRJ4UFoCeQ,branch~uRNpPAOHEe-PZgr_4p5lOw,Live,False,erYuK,Sinusoidal Steady State Analysis,TigIx,Applying Network Theorems in AC Circuits,"In this module, you will be able to represent ...",ckYYK,Superposition Theorem in AC Circuits,reading,103,6,17,124,"<co-content><text variant=""body1""><strong>Read..."
54928,zzjnW9fWEe628RJzOJU3jw,1,uBPFd1T3Eeqa6Ar_L1S13Q,branch~yy9FstfWEe6PnQom9vUvGw,Upcoming,False,IgEdm,Week 4,U80Z3,Areal Data: Smoothing and Finding Hot Spots,In this unit we will discuss the following top...,eJWqo,Graham (2008): Intelligent smoothing using hie...,reading,103,4,6,81,<co-content><text>Please read:</text><text>P. ...


## Create GPT Client

In [4]:
from openai import AzureOpenAI
from tenacity import retry, stop_after_attempt, wait_fixed

API_BASE = "https://makeathon2024a9363557799.openai.azure.com/"
API_VERSION = "2024-02-01"
API_KEY = '577dc66c1c924b2e9287962f161a8913'

class OpenAIClient:

    def __init__(self, system_prompt):
        self.system_prompt = system_prompt
        self.client = AzureOpenAI(
          azure_endpoint = API_BASE, 
          api_key=API_KEY,  
          api_version="2024-02-15-preview"
        )

    @retry(stop=stop_after_attempt(10), wait=wait_fixed(1))
    def create_completion(self, user_prompt):
        message = [
            {
                "role":"system",
                "content": self.system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]

        return self.client.chat.completions.create(
          model="gpt-4",
          messages = message,
          temperature=0.7,
          max_tokens=800,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )

In [5]:
SYSTEM_PROMPT = \
"""
Please identify any outdated information in the following content. Return a json string with the following format:

{
   "resultList": [
      "content": <OUTDATED_CONTENT>,
      "recommendedReplacement": <RECOMMENDED_REPLACEMENT>,
      "reason": <REASON>
   ]
}

No outdated content:

{}
"""

In [6]:
client = OpenAIClient(SYSTEM_PROMPT)

In [7]:
result = client.create_completion("this is a test")

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

class DataAIEnricher:
    def __init__(self, client):
        self.client = client

    def enrich_single(self, row, column: str):
        content = row[column]
        result = self.client.create_completion(content)
        return result

    def enrich(self, df: pd.DataFrame, column: str, new_col: str) -> []:
        res = []
        with ThreadPoolExecutor() as executor:
            futures = {executor.submit(self.enrich_single, row, column): row for _, row in df.iterrows()}

            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rows"):
                row = futures[future]
                try:
                    result = future.result()
                    res.append(response.choices[0].message.content)
                except Exception as exc:
                    print(f'Exception for row {row}: {exc}')
                    res.append("{}")
        df[new_col] = res
        return df


In [9]:
enricher = DataAIEnricher(client)

In [10]:
enriched_df = enricher.enrich(df.head(10), 'reading_content', 'outdated_recommendations')

Processing rows:  10%|██▋                        | 1/10 [00:00<00:04,  1.87it/s]

Exception for row atom_id                                                     -0UecqDOEeuJPxL-WbpBJQ
atom_version_id                                                                  6
course_id                                                   TMggHmbLEeuxXgqV68ldrQ
course_branch_id                                            TMggHmbLEeuxXgqV68ldrQ
course_branch_status                                                          Live
is_course_branch_default                                                      True
course_module_id                                                             OiYaf
course_module_name                                           Descriptive Analytics
course_lesson_id                                                             gri4U
course_lesson_name                                                     Module Quiz
course_module_desc               This lecture covers the first steps to analysi...
course_item_id                                                       

Processing rows:  60%|████████████████▏          | 6/10 [00:06<00:04,  1.08s/it]

Exception for row atom_id                                                     -2rrU7s9Eeydqwrxl_qHrw
atom_version_id                                                                  2
course_id                                                   6_cTrLs9Eey8ogoy0lwspQ
course_branch_id                                            6_cTrLs9Eey8ogoy0lwspQ
course_branch_status                                                          Live
is_course_branch_default                                                      True
course_module_id                                                             lJvtj
course_module_name                                          Présentation du projet
course_lesson_id                                                             8dDTs
course_lesson_name                                  Votre parcours d'apprentissage
course_module_desc               Ce projet guidé “Gérer les fichiers et réperto...
course_item_id                                                       

Processing rows:  70%|██████████████████▉        | 7/10 [00:08<00:03,  1.27s/it]

...

Processing rows:  90%|████████████████████████▎  | 9/10 [00:13<00:01,  1.72s/it]

Exception for row atom_id                                                     --e_h9tbEe628RJzOJU3jw
atom_version_id                                                                  1
course_id                                                   yfMErXgQEe2QHRL3YiV0EQ
course_branch_id                                     branch~8sgWU9tbEe6xnw6R70Kdnw
course_branch_status                                                          Live
is_course_branch_default                                                     False
course_module_id                                                             kSa73
course_module_name                  Maximizando la Inclusión con Simulaciones PhET
course_lesson_id                                                             zSLcM
course_lesson_name               Funciones Inclusivas para Diversas Necesidades...
course_module_desc               En este módulo, nos enfocamos en cultivar un e...
course_item_id                                                       

Processing rows: 100%|██████████████████████████| 10/10 [00:16<00:00,  1.62s/it]

Exception for row atom_id                                                     -2wVv9bFEe6rqw6KY2NYGQ
atom_version_id                                                                  1
course_id                                                   3oedlEG8Ee2hOhI8guE3RQ
course_branch_id                                     branch~9zh-JdbFEe6lEg6XAETioQ
course_branch_status                                                      Upcoming
is_course_branch_default                                                     False
course_module_id                                                             g96SY
course_module_name                                           Discriminant Analysis
course_lesson_id                                                             ZsZue
course_lesson_name                                  Discriminant Analysis: Concept
course_module_desc               In this module, you will learn about discrimin...
course_item_id                                                       


/var/folders/bz/c2jh87dj6rs5m2xxply8xkq00000gp/T/ipykernel_79715/3191504398.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = res


In [11]:
enriched_df.to_csv(SAVE_FILENAME)